In [75]:
# 25/01/2024

# Topic Modeling
# LDA (Latent Dirichlet Allocation): 
# More info: 
# https://towardsdatascience.com/latent-dirichlet-allocation-lda-9d1cd064ffa2
# https://blog.echen.me/2011/08/22/introduction-to-latent-dirichlet-allocation/
# https://towardsdatascience.com/topic-modeling-and-latent-dirichlet-allocation-in-python-9bf156893c24

# Topic modeling is a method for unsupervised classification of documents, similar to clustering
# on numeric data, which finds some natural groups of items (topics) even when we’re not sure what we’re looking for.
# LDA is one of the most popular topic modeling methods

# Taller de PLN - USAR LDA

Este ejemplo está diseñado para principiantes y se centra en el uso de la biblioteca gensim para aplicar LDA. Asegúrate de que gensim esté instalado en tu entorno de Python antes de ejecutar el código.

Este ejemplo asumirá que tienes un conjunto de noticias en español en formato de texto, aunque podrías aplicar el mismo enfoque a otros tipos de textos. El objetivo es identificar temas latentes en el corpus de noticias utilizando LDA.

Este código primero realiza el preprocesamiento de texto en el corpus de noticias, crea un diccionario y una representación de documento-bolsa de palabras, luego aplica el modelo LDA para identificar temas latentes y visualiza los resultados utilizando pyLDAvis.

Este ejemplo proporciona una introducción básica al uso de LDA para identificar temas en un corpus de texto en español y está diseñado para principiantes en NLP y procesamiento de lenguaje natural. Puedes personalizar y ampliar este código según tus necesidades y el conjunto de datos específico que estés utilizando.

## Paso 1: Instalar las bibliotecas necesarias (asegúrate de tener gensim, nltk y pyLDAvis instalados):

In [76]:
# pip install gensim nltk pyLDAvis

### Importar bibliotecas a utilizar

In [77]:
import gensim
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import pyLDAvis.gensim
from gensim import corpora
from gensim.models import LdaModel
import re
import pyLDAvis.gensim

In [78]:
# Descargar los recursos de NLTK necesarios
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\AlejandroDiezRedondo\AppData\Roaming\nltk_dat
[nltk_data]     a...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\AlejandroDiezRedondo\AppData\Roaming\nltk_dat
[nltk_data]     a...
[nltk_data]   Package stopwords is already up-to-date!


True

## Paso 2: Importar las bibliotecas y cargar el corpus de noticias.

Los temas que deberían salir:

- Política y Gobierno: Dado que hay noticias relacionadas con el gobierno, la inversión en infraestructura y la legislación de inmigración, es probable que emerjan temas relacionados con la política y la gobernanza.

- Medio Ambiente y Cambio Climático: Las noticias sobre la caída de los mercados bursátiles, el cambio climático y las protestas por el medio ambiente pueden dar lugar a temas relacionados con la sostenibilidad y el medio ambiente.

- Tecnología y Ciencia: Las menciones a Apple, investigaciones científicas y avances tecnológicos sugieren la presencia de temas relacionados con la tecnología y la ciencia.

- Cultura y Entretenimiento: Las noticias sobre películas, arte contemporáneo y música indican temas culturales y de entretenimiento.

- Deportes: La referencia al Real Madrid y la clasificación de la selección de fútbol femenino para los Juegos Olímpicos apunta a temas deportivos.

- Salud y Bienestar: La relación entre el consumo de café y la longevidad, así como las noticias sobre el cáncer, pueden generar temas relacionados con la salud y el bienestar.

- Economía y Negocios: Las noticias sobre inversión en infraestructura, regulaciones de privacidad y la caída de los mercados bursátiles podrían dar lugar a temas económicos y empresariales.

- Exploración Espacial: La mención de una sonda espacial española y un exoplaneta podría conducir a temas relacionados con la exploración espacial y la astronomía.

- Educación: La inversión en educación y las reformas pueden generar temas relacionados con la educación y la pedagogía.

In [79]:
# Cargar el corpus de noticias en español
corpus = [
    "El gobierno anuncia un nuevo plan de inversión en infraestructura.",
    "Investigadores descubren una nueva especie de mariposa en la Amazonía.",
    "Los mercados bursátiles experimentan una caída en medio de la incertidumbre económica.",
    "Apple presenta su último modelo de iPhone con características innovadoras.",
    "Estudio revela vínculos entre el consumo de café y la longevidad.",
    "Una película española gana el premio a la mejor película en el Festival de Cine de Cannes.",
    "El Real Madrid celebra su séptimo campeonato consecutivo de liga.",
    "Protestas masivas exigen medidas concretas para combatir el cambio climático.",
    "El gobierno anuncia una inversión récord en la educación pública.",
    "Científicos españoles lideran un proyecto de investigación para encontrar la cura del cáncer.",
    "Netflix lanza una nueva serie de ciencia ficción muy esperada.",
    "Nuevas regulaciones de privacidad en línea generan un debate sobre la protección de datos.",
    "Importante exposición de arte contemporáneo llega a una galería de renombre.",
    "La pandemia de COVID-19 afecta la economía global y plantea desafíos en la atención médica.",
    "Artista español de renombre recibe un premio por su contribución a la cultura.",
    "La selección de fútbol femenino de España se clasifica para los Juegos Olímpicos.",
    "Lanzamiento de una sonda espacial española para explorar un exoplaneta recién descubierto.",
    "Nueva legislación de inmigración busca reformar el sistema de asilo.",
    "Hallazgo arqueológico revela los secretos de una antigua civilización en la península ibérica.",
    "Gira internacional de una banda de música española cosecha éxitos en todo el mundo.",
    "Desarrollo de vehículos eléctricos en España gana impulso con el lanzamiento de nuevos modelos."
]

# Preprocesamiento de texto
def preprocess_text(text):
    text = re.sub(r'\d+', '', text)  # Eliminar números
    text = text.lower()  # Convertir a minúsculas
    text = re.sub(r'[^\w\s]', '', text)  # Eliminar puntuación
    words = word_tokenize(text)  # Tokenización de palabras

    '''
    list comprehension:
    The 1º word is the expression that gets included in the new list.
    The 2º word is the variable that takes each value from the iterable (words in this case) during each iteration of the loop.
    '''
    words = [word for word in words if word not in stopwords.words('spanish')]  # Eliminar palabras de parada en español

    return words

# Aplicar preprocesamiento a cada noticia
processed_corpus = [preprocess_text(news) for news in corpus]

In [80]:
i = 0

for processed_news in processed_corpus:
    print(i, processed_news)
    i += 1

0 ['gobierno', 'anuncia', 'nuevo', 'plan', 'inversión', 'infraestructura']
1 ['investigadores', 'descubren', 'nueva', 'especie', 'mariposa', 'amazonía']
2 ['mercados', 'bursátiles', 'experimentan', 'caída', 'medio', 'incertidumbre', 'económica']
3 ['apple', 'presenta', 'último', 'modelo', 'iphone', 'características', 'innovadoras']
4 ['estudio', 'revela', 'vínculos', 'consumo', 'café', 'longevidad']
5 ['película', 'española', 'gana', 'premio', 'mejor', 'película', 'festival', 'cine', 'cannes']
6 ['real', 'madrid', 'celebra', 'séptimo', 'campeonato', 'consecutivo', 'liga']
7 ['protestas', 'masivas', 'exigen', 'medidas', 'concretas', 'combatir', 'cambio', 'climático']
8 ['gobierno', 'anuncia', 'inversión', 'récord', 'educación', 'pública']
9 ['científicos', 'españoles', 'lideran', 'proyecto', 'investigación', 'encontrar', 'cura', 'cáncer']
10 ['netflix', 'lanza', 'nueva', 'serie', 'ciencia', 'ficción', 'esperada']
11 ['nuevas', 'regulaciones', 'privacidad', 'línea', 'generan', 'debate', 

## Paso 3: Crear un diccionario y una representación de documento-bolsa de palabras.

In [81]:
# Crear un diccionario basado en el corpus preprocesado
dictionary = corpora.Dictionary(processed_corpus)

# Crear una representación de documento-bolsa de palabras
corpus_bow = [dictionary.doc2bow(text) for text in processed_corpus]

In [82]:
print(dictionary)

Dictionary<141 unique tokens: ['anuncia', 'gobierno', 'infraestructura', 'inversión', 'nuevo']...>


In [83]:
print(corpus_bow)

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1)], [(6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1)], [(12, 1), (13, 1), (14, 1), (15, 1), (16, 1), (17, 1), (18, 1)], [(19, 1), (20, 1), (21, 1), (22, 1), (23, 1), (24, 1), (25, 1)], [(26, 1), (27, 1), (28, 1), (29, 1), (30, 1), (31, 1)], [(32, 1), (33, 1), (34, 1), (35, 1), (36, 1), (37, 1), (38, 2), (39, 1)], [(40, 1), (41, 1), (42, 1), (43, 1), (44, 1), (45, 1), (46, 1)], [(47, 1), (48, 1), (49, 1), (50, 1), (51, 1), (52, 1), (53, 1), (54, 1)], [(0, 1), (1, 1), (3, 1), (55, 1), (56, 1), (57, 1)], [(58, 1), (59, 1), (60, 1), (61, 1), (62, 1), (63, 1), (64, 1), (65, 1)], [(11, 1), (66, 1), (67, 1), (68, 1), (69, 1), (70, 1), (71, 1)], [(72, 1), (73, 1), (74, 1), (75, 1), (76, 1), (77, 1), (78, 1), (79, 1)], [(80, 1), (81, 1), (82, 1), (83, 1), (84, 1), (85, 1), (86, 1)], [(87, 1), (88, 1), (89, 1), (90, 1), (91, 1), (92, 1), (93, 1), (94, 1), (95, 1)], [(39, 1), (86, 1), (96, 1), (97, 1), (98, 1), (99, 1), (100, 1)], [(101, 1), (102, 1

## Paso 4: Aplicar el modelo LDA y explorar los temas latentes.

In [84]:
# Entrenar un modelo LDA con 3 temas (ajusta el número según sea necesario)
lda_model = LdaModel(corpus=corpus_bow, num_topics=3, id2word=dictionary, passes=15)

In [85]:
# Imprimir los temas y sus palabras clave (cambian en cada run all)
topics = lda_model.print_topics(num_words=5)
for topic in topics:
    print(topic)

(0, '0.026*"inversión" + 0.026*"gobierno" + 0.026*"anuncia" + 0.015*"española" + 0.015*"renombre"')
(1, '0.024*"nueva" + 0.024*"lanzamiento" + 0.013*"gana" + 0.013*"españa" + 0.013*"española"')
(2, '0.022*"película" + 0.012*"revela" + 0.012*"premio" + 0.012*"exigen" + 0.012*"cine"')


In [86]:
# Visualizar los temas con pyLDAvis
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus_bow, dictionary)

In [87]:
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2     -0.092737  0.020894       1        1  38.746251
1      0.026885 -0.085036       2        1  33.555786
0      0.065852  0.064142       3        1  27.697964, topic_info=            Term      Freq     Total Category  logprob  loglift
3      inversión  1.000000  1.000000  Default  30.0000  30.0000
1       gobierno  1.000000  1.000000  Default  29.0000  29.0000
0        anuncia  1.000000  1.000000  Default  28.0000  28.0000
112  lanzamiento  1.000000  1.000000  Default  27.0000  27.0000
38      película  1.000000  1.000000  Default  26.0000  26.0000
..           ...       ...       ...      ...      ...      ...
82    exposición  0.642632  1.003518   Topic3  -4.2018   0.8381
107    selección  0.642631  1.003517   Topic3  -4.2018   0.8381
34      española  0.644422  2.084864   Topic3  -4.1990   0.1097
86      renombre  0.643609  1.528750   Topic3  -4.2003   0.4187
102       españa  0.642671  1.528823   Topic3  -4.2017   0.4172

[130 rows x 6 columns], token_table=      Topic      Freq          Term
term                               
87        2  0.955510        afecta
6         2  0.955503      amazonía
121       1  0.928038       antigua
0         3  0.673068       anuncia
122       1  0.928038  arqueológico
...     ...       ...           ...
71        2  0.955501         serie
114       2  0.955502         sonda
140       2  0.955500     vehículos
31        2  0.955502      vínculos
134       3  0.996498        éxitos

[105 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 2, 1])

## Conclusiones

---- Algunas palabras claves cambian en cada Run All ----

A continuación, se presentan las interpretaciones de cada uno de estos temas:

- Tema 0 - Tema de Entretenimiento y Cultura:
Este tema parece estar relacionado con noticias de entretenimiento y cultura. Las palabras clave "española", "lanzamiento", "gana" y "película" sugieren que este tema podría centrarse en películas españolas y su éxito, incluido el lanzamiento de una película que ha ganado un premio. También se mencionan vehículos eléctricos, lo que podría estar relacionado con películas o documentales sobre la adopción de vehículos eléctricos en la cultura contemporánea.

- Tema 1 - Tema de Política y Gobierno:
Este tema parece estar relacionado con noticias de política y gobierno. Las palabras clave "nueva", "inversión", "gobierno" y "anuncia" indican que se trata de anuncios gubernamentales de nuevas inversiones o políticas. Es posible que este tema aborde temas como reformas gubernamentales, inversiones en infraestructura y anuncios políticos relevantes.

- Tema 2 - Tema de Arqueología e Historia:
Este tema parece estar relacionado con noticias relacionadas con la arqueología e historia. Las palabras clave "revela", "España", "renombre", "ibérica" y "secretos" sugieren que se trata de noticias que revelan secretos históricos o descubrimientos en España. Esto podría incluir hallazgos arqueológicos, investigaciones sobre la historia de la península ibérica y eventos relacionados con el patrimonio cultural y arqueológico.

Es importante recordar que estas interpretaciones se basan en las palabras clave identificadas por el algoritmo LDA y pueden no ser completamente precisas. La interpretación de los temas es un proceso subjetivo y requiere un conocimiento contextual más profundo para comprender completamente el significado de cada tema. El ajuste de hiperparámetros y la exploración adicional del corpus pueden mejorar la precisión de las interpretaciones de los temas latentes.